In [ ]:
#testing conumption of stock stream data
from confluent_kafka import Consumer, KafkaError

def consume_stock_data(topic, num_messages):
    conf = {
        'bootstrap.servers': '145.191.53.50:9092',
        'group.id': 'stock-data-consumer',
        'auto.offset.reset': 'earliest'
    }

    consumer = Consumer(conf)
    consumer.subscribe([topic])

    messages = []
    for _ in range(num_messages):
        message = consumer.poll(1.0)
        if message is None:
            continue
        if message.error():
            if message.error().code() == KafkaError._PARTITION_EOF:
                # End of partition event - not an error.
                print(f'Reached end of partition {message.partition()}.')
            else:
                print(f'Error while consuming message: {message.error()}')
        else:
            # Proper message
            messages.append(message.value().decode('utf-8'))

    consumer.close()
    return messages

# Consume messages from the 'stock-streaming-topic' topic
num_messages_to_fetch = 100  # You can change this to the number of messages you wish to fetch
messages = consume_stock_data('stock-streaming-topic', num_messages_to_fetch)

# Print out the content of the received stock data messages
for msg in messages:
    print(msg)
